In [1]:
import json
from itertools import cycle
from collections import defaultdict
import uuid
import numpy as np
from vizbuilder import Html, components, require

In [2]:
colors_iter = cycle(["red", "blue", "green", "orange", "purple"])
colors = defaultdict(lambda: next(colors_iter))

rand = np.random.RandomState(42)
data = [
        {"category": k, "subcategory": j, "start": int(i), "duration": rand.choice(20), "color": colors[k]}
        for k in ["A", "B", "C", "D", "E"] for j in range(3) for i in np.cumsum(rand.choice(20, 20) + 20) 
    ]

In [3]:
# Original Gantt Chart (without scrubber for comparison)
container_id = f"d3-container-{uuid.uuid4().hex[:8]}"   
user_code = f"""
    const data={json.dumps(data)};
    const container = d3.select("#{container_id}").datum(data);
    const chart = ganttChart()
        .axisFontSize(18)                    // Larger axis labels
        .titleFontSize(22)                   // Larger title
        .title(null)        // Custom title
        .xAxisLabel("Cycles")                  // Custom x label
        .yAxisLabel(null)                    // Hide y label
        .showGridLines(false)                // Clean look
        .categoryBackgroundOpacity(0.5, 0.1) // Bolder groups
        .interactiveMode(false)              // Static export
        .colorScheme(  ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"]); // Custom colors
          
    //.scrubberEnabled(false)
    //;  // Disable scrubber for this demo
    container.call(chart);
    window.addEventListener('resize', (() => container.call(chart)));
"""

html = Html.body(
    Html.h3("Basic Gantt Chart (no scrubber)"),
    Html.div(id_=container_id),
    Html.script(require(modules=["d3", "ganttChart"], user_code=user_code, clear_cache=True))
)
display(html)

In [4]:
# Enhanced Gantt Chart with Interactive Scrubber and Linked Data Display
chart_container_id = f"chart-container-{uuid.uuid4().hex[:8]}"
display_container_id = f"display-container-{uuid.uuid4().hex[:8]}"

user_code = f"""
    //console.log("Starting to load modules...");
    
    const data={json.dumps(data)};
    
    try {{
        // Test if modules are available
        console.log("d3 available:", typeof d3);
        console.log("ganttChart available:", typeof ganttChart);
        console.log("dataDisplay available:", typeof dataDisplay);
        
        // Create the linked data display component first
        const displayContainer = d3.select("#{display_container_id}");
        const displayComponent = dataDisplay().title("Timeline Entries at Scrubber Position");
        displayContainer.call(displayComponent);
        //console.log("Data display component created successfully");
        
        // Create the Gantt chart with scrubber enabled
        const chartContainer = d3.select("#{chart_container_id}").datum(data);
        const chart = ganttChart().scrubberEnabled(true);
        
        // Link the components using the event system BEFORE rendering chart
        chart.on("scrubberMove", function(event) {{
            // console.log("Scrubber moved, intersected data:", event.intersectedData);
            // Update the data display with intersected entries
            const displayNode = displayContainer.node();
            displayNode.updateData(event.intersectedData);
        }});
        
        // Now render the chart (this will emit the initial scrubberMove event)
        chartContainer.call(chart);
        //console.log("Gantt chart created successfully");
        
        // Handle window resize for chart
        window.addEventListener('resize', (() => chartContainer.call(chart)));
        
        //console.log("All components initialized successfully");
        
    }} catch (error) {{
        console.error("Error in chart initialization:", error);
        console.error("Error stack:", error.stack);
    }}
"""

html = Html.body(
    Html.h3("Interactive Gantt Chart with Scrubber"),
    Html.p("Drag the orange scrubber line to see which timeline entries intersect. The data display below updates automatically."),
    Html.div(id_=chart_container_id, style="height: 350px; margin-bottom: 20px;"),
    Html.div(id_=display_container_id, style="height: 300px;"),
    Html.script(require(modules=["d3", "ganttChart", "dataDisplay"], user_code=user_code, clear_cache=True))
)
display(html)

In [5]:
# Gantt Chart with Excalidraw Virgil Font
# This example shows how to use the exact Excalidraw handwritten font for titles and axes

container_id = f"d3-container-excalidraw-{uuid.uuid4().hex[:8]}"   
user_code = f"""
    const data={json.dumps(data)};
    const container = d3.select("#{container_id}").datum(data);
    const chart = ganttChart()
        .axisFontSize(14)
        .titleFontSize(24)
        .title("Project Timeline")
        .xAxisLabel("Time (cycles)")
        .yAxisLabel("Teams")
        .showGridLines(true)
        .categoryBackgroundOpacity(0.3, 0.05)
        .interactiveMode(true)
        .scrubberEnabled(true)
        .colorScheme(["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"])
        // Configure Excalidraw's Virgil font
        .titleFontFamily("Virgil, Segoe UI Emoji")
        .axisFontFamily("Virgil, Segoe UI Emoji") 
        .fontUrl("https://excalidraw.com/Virgil.woff2");
          
    container.call(chart);
    window.addEventListener('resize', (() => container.call(chart)));
"""

html = Html.body(
    Html.h3("Gantt Chart with Excalidraw Virgil Font"),
    Html.p("This chart uses Excalidraw's Virgil font for the title and axis labels, matching the handwritten style of Excalidraw annotations."),
    Html.div(id_=container_id, style="height: 400px;"),
    Html.script(require(modules=["d3", "ganttChart"], user_code=user_code, clear_cache=True))
)
display(html)

## Custom Font Support

The Gantt chart now supports custom fonts for titles and axis labels. This is particularly useful when you want the chart to match annotations you'll add manually (e.g., using Excalidraw).

### Font Configuration Options:

- **`.titleFontFamily(fontFamily)`**: Set the font family for the chart title
- **`.axisFontFamily(fontFamily)`**: Set the font family for axis labels and tick labels
- **`.fontUrl(url)`**: URL to load a custom font file (WOFF2 format recommended)

### Using Excalidraw's Virgil Font:

Excalidraw uses the "Virgil" font for its handwritten style. To make your chart match Excalidraw annotations exactly:

```javascript
chart
    .titleFontFamily("Virgil, Segoe UI Emoji")
    .axisFontFamily("Virgil, Segoe UI Emoji")
    .fontUrl("https://excalidraw.com/Virgil.woff2");
```

The font is loaded dynamically and applied to:
- Chart title
- X-axis label
- Y-axis label  
- X-axis tick labels
- Y-axis category labels

**Note**: The exact Excalidraw font is required for matching - similar fonts won't align with manual annotations.